In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp

In [2]:
## changing the refinary with the depot_farm location

## importing _data

In [3]:
distance = pd.read_excel('Disttt.xlsx')
forecast_2018 = pd.read_excel('pred_18.xlsx')
forecast_2019 = pd.read_excel('pred_19.xlsx')

In [4]:
## reorganizing
distance = distance.drop(columns='Unnamed: 0')
forecast_2018 = forecast_2018.drop(columns='Unnamed: 0')
forecast_2019 = forecast_2019.drop(columns='Unnamed: 0')
forecast_2018.set_index('Index',drop=True,inplace=True)
forecast_2019.set_index('Index',drop=True,inplace=True)

In [5]:
#keys
farm = distance.index
farm_depot = distance.index
depot_refinary = distance.index
refinary = distance.index
distance.loc[5,7]

21.5222999999999

In [6]:
farm_depot_keys = [(f,d) for f in farm for d in farm_depot]
depot_refinary_keys = [(d,r) for d in depot_refinary for r in refinary ]

In [7]:
dist_dict = {(f,d): distance.loc[f,d] for f in farm for d in farm_depot}
dist_dict.get((5, 7))

21.5222999999999

In [8]:
# Define the Gurobi model
model = gp.Model('optimize')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-14


In [9]:
a = 0.001
b = 1
c = 1

In [10]:
# Define the variables
open_depot = { d : model.addVar(vtype=gp.GRB.BINARY, name=f'open_depot_{d}')  for d in farm_depot}
open_refinary = { d : model.addVar(vtype=gp.GRB.BINARY, name=f'open_refinary_{d}') for d in depot_refinary}
biomass_flows = {(f, d): model.addVar(lb=0, vtype=gp.GRB.CONTINUOUS, name=f'biomass_flows_{f}_{d}') for (f, d) in farm_depot_keys}
pellet_flows = {(d, r): model.addVar(lb=0, vtype=gp.GRB.CONTINUOUS, name=f'pellet_flows_{d}_{r}') for (d, r) in depot_refinary_keys}

In [11]:
# Set the objective function
Transport_cost_biomass = gp.quicksum(a * dist_dict.get((f, d)) * biomass_flows.get((f, d)) for f in farm for d in farm_depot )
Transport_cost_pellet = gp.quicksum(a * dist_dict.get((d,r)) * pellet_flows.get((d, r)) for d in depot_refinary for r in refinary)

In [12]:
Underutilization_cost_biomass = gp.quicksum(open_depot.get(d) * 200 for d in farm_depot) - gp.quicksum(biomass_flows.get((f, d)) for f in farm for d in farm_depot) 
Underutilization_cost_pellet = gp.quicksum(open_refinary.get(d) * 700 for d in depot_refinary) - gp.quicksum(pellet_flows.get((d, r)) for d in depot_refinary for r in refinary)

In [13]:
cost_forecast = sum(forecast_2018['mae'])

In [14]:
model.setObjective(Transport_cost_biomass + Transport_cost_pellet + cost_forecast +Underutilization_cost_biomass+Underutilization_cost_pellet , gp.GRB.MINIMIZE)

In [15]:
model.addConstr(Transport_cost_biomass + Transport_cost_pellet + cost_forecast +Underutilization_cost_biomass+Underutilization_cost_pellet >= 0, name='main')

<gurobi.Constr *Awaiting Model Update*>

In [16]:
# Add constraints
for f in farm:
    for d in farm_depot:
        model.addConstr(biomass_flows.get((f,d)) >= 0 ,name =f'bio_flow_{f}_{d}')

In [17]:
for d in depot_refinary:
    for r in refinary:
        model.addConstr(pellet_flows.get((d,r)) >= 0 , name = f'pel_flow_{d}_{r}')


In [18]:
for d in farm_depot:
    model.addConstr(gp.quicksum(biomass_flows.get((f, d)) for f in farm) <=  200, name=f'depot_{d}_limit')

In [19]:
for r in refinary:
    model.addConstr(gp.quicksum( pellet_flows.get((d, r)) for d in depot_refinary) <= 700, name=f'refinery_{r}_limit')

In [20]:
#for d in farm_depot:
 #   for f in farm:
  #      model.addConstr(biomass_flows.get((f,d)) <= open_depot.get(d) * 200 , name=f'depot_{f}_{d}_link')

In [21]:
#for d in depot_refinary:
 #  for r in refinary:
   #         model.addConstr(pellet_flows.get((d, r)) <= open_refinary.get(d) * 700 , name=f'refinary_{f}_{d}_link')

In [22]:
model.addConstr(gp.quicksum(open_depot.get(d) for d in farm_depot) >= 15, name='depot_limit_lower')
model.addConstr(gp.quicksum(open_refinary.get(d) for d in depot_refinary) >= 2, name='refinery_limit_lower')

<gurobi.Constr *Awaiting Model Update*>

In [23]:
model.addConstr(gp.quicksum(open_depot.get(d) for d in farm_depot) <= 20, name='depot_limit_upper')
model.addConstr(gp.quicksum(open_refinary.get(d) for d in depot_refinary) <= 5, name='refinery_limit_upper')

<gurobi.Constr *Awaiting Model Update*>

In [24]:
for f in farm:
    model.addConstr(gp.quicksum(open_depot.get(d) * biomass_flows.get((f, d)) for d in farm_depot) >= 0.8 * forecast_2018['forecast'].loc[f], name=f'farm_{f}_lower_limit')

In [25]:
for f in farm:
    model.addConstr(gp.quicksum(biomass_flows.get((f, d)) for d in farm_depot) <= forecast_2018['forecast'].loc[f], name=f'farm_{f}_upper_limit')

In [26]:
for f in farm:
    for d in farm_depot:
        model.addConstr(biomass_flows.get((f, d)) <= open_depot.get(d) * forecast_2018['forecast'].loc[f], name=f'link_{f}_{d}_bio')


In [27]:
#for f in farm:
#    for d in farm_depot:
#        model.addConstr(biomass_flows.get((f, d)) <= open_depot.get(d) * forecast_2018['forecast'].loc[f], name=f'farm_{f}depot{d}_biomass_constraint')

In [28]:
#for f in farm:
#    for d in farm_depot:
#        model.addConstr(biomass_flows.get((f, d)) <= open_depot.get(d), name=f'biomass_to_open_depot_{f}_{d}')

In [29]:
for d in farm_depot:
    model.addConstr(gp.quicksum(biomass_flows[i, d] for i in farm) == gp.quicksum(open_refinary.get(r)*pellet_flows[d,r] for r in refinary), name=f'balance_depot_{d}')

In [30]:
model.addConstr(gp.quicksum( biomass_flows.get((f, d)) for f in farm for d in farm_depot) == gp.quicksum( pellet_flows.get((d, r))for d in depot_refinary for r in refinary), name=f'balance_{d}')

<gurobi.Constr *Awaiting Model Update*>

In [31]:
#for d in farm_depot:
#    model.addConstr(
#        gp.quicksum(biomass_flows.get((i, d)) for i in farm) == gp.quicksum(pellet_flows.get((d, r)) for r in refinery),
#        name=f'balance_depot_{d}_biomass_pellet',
#    )
#    # Adding a tolerance constraint
#    model.addConstr(
 #       gp.quicksum(biomass_flows.get((i, d)) for i in farm)
#        - gp.quicksum(pellet_flows.get((d, r)) for r in refinery)
#        <= 1e-03,
#        name=f'balance_tolerance_depot_{d}',
#    )

In [32]:
for d in farm_depot:
#    for r in refinary:
        model.addConstr(open_depot.get(d) + open_refinary.get(d) <= 1, name=f'one_depot_{d}')

## model optimiaztion

In [33]:
#model.setParam('NodefileStart', 0.1)
model.setParam("MIPGap",1)

Set parameter MIPGap to value 1


In [34]:
model.setParam("MIPFocus", 2)

Set parameter MIPFocus to value 2


In [35]:
model.setParam("Heuristics",1)

Set parameter Heuristics to value 1


In [36]:
model.setParam("Presolve", 2)

Set parameter Presolve to value 2


In [37]:
#model.setParam("Threads", 8) 

In [38]:
#def myLazyCallback(model, where):
    #if where == gp.GRB.Callback.MIPSOL:
        # Add lazy constraints if needed based on the current solution
       # pass

#model.setCallback(myLazyCallback)

In [39]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8013 rows, 5304 columns and 29019 nonzeros
Model fingerprint: 0x689d9db0
Model has 102 quadratic constraints
Variable types: 5202 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e-01, 7e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [8e-01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 7e+02]
  QRHS range       [1e-01, 1e+02]
Presolve removed 5202 rows and 0 columns
Presolve time: 0.08s
Presolved: 15918 rows, 10506 columns, 60280 nonzeros
Variable types: 10404 continuous, 102 integer (102 binary)
Root relaxation presolve removed 2 rows and 0 columns
Root relaxation presolved: 15814 rows, 10457 columns, 59874 nonzeros


Root relaxation: objective 1.5

In [40]:
open_depot_values = {var.varName: var.x for var in open_depot.values()}

In [41]:
for var_name, var_value in open_depot_values.items():
    if var_value == 1:
        print(f"{var_name}: {var_value}")

open_depot_2: 1.0
open_depot_3: 1.0
open_depot_5: 1.0
open_depot_7: 1.0
open_depot_11: 1.0
open_depot_12: 1.0
open_depot_21: 1.0
open_depot_31: 1.0
open_depot_33: 1.0
open_depot_35: 1.0
open_depot_36: 1.0
open_depot_38: 1.0
open_depot_39: 1.0
open_depot_40: 1.0
open_depot_47: 1.0
open_depot_49: 1.0
open_depot_50: 1.0


In [42]:
open_refinary_values = {var.varName: var.x for var in open_refinary.values()}

In [43]:
for var_name, var_value in open_refinary_values.items():
    if var_value == 1:
        print(f"{var_name}: {var_value}")

open_refinary_1: 1.0
open_refinary_10: 1.0
open_refinary_15: 1.0
open_refinary_17: 1.0
open_refinary_23: 1.0


In [44]:
#*open_depot.get(d) *open_refinary.get(r)

In [45]:
biomass_flow = {var.varName: var.x for var in biomass_flows.values()}

In [46]:
for var_name, var_value in biomass_flow.items():
    if var_value > 0:
        print(f"{var_name}: {var_value}")

biomass_flows_0_31: 5.929175694245551
biomass_flows_1_2: 11.820901839571349
biomass_flows_1_49: 29.463641579715905
biomass_flows_2_2: 66.1755162784214
biomass_flows_3_3: 92.2309861683627
biomass_flows_4_5: 22.6553668642659
biomass_flows_5_3: 33.8686852567464
biomass_flows_6_7: 48.18742094217309
biomass_flows_6_19: 2.220446049250313e-16
biomass_flows_7_7: 40.6158806280898
biomass_flows_8_7: 6.2928626519930795
biomass_flows_8_11: 25.17145060797232
biomass_flows_9_11: 41.3166513031641
biomass_flows_10_11: 8.10905808765618
biomass_flows_11_11: 7.27440046391346
biomass_flows_12_11: 6.75178581344663
biomass_flows_13_11: 0.124837599106508
biomass_flows_14_47: 0.521874729980293
biomass_flows_15_15: 4.132516551180743e-11
biomass_flows_15_49: 23.45952637900168
biomass_flows_16_2: 2.3583837889606523
biomass_flows_16_15: 4.132516551180743e-11
biomass_flows_16_50: 89.15963778731262
biomass_flows_17_2: 111.5168404413082
biomass_flows_17_33: 38.12959640161678
biomass_flows_18_3: 69.48556560321228
bio

In [47]:
pellet_flow_values = {var.varName: var.x for var in pellet_flows.values()}

In [48]:
for var_name, var_value in pellet_flow_values.items():
    if var_value > 0:
        print(f"{var_name}: {var_value}")

pellet_flows_2_1: 200.000000000015
pellet_flows_3_1: 194.45529018253168
pellet_flows_3_17: 5.544709817468288
pellet_flows_5_17: 200.0
pellet_flows_7_23: 200.000000000015
pellet_flows_11_10: 200.0
pellet_flows_12_10: 194.4552901825167
pellet_flows_14_29: 1.3210694760346087e-09
pellet_flows_21_23: 200.000000000015
pellet_flows_31_1: 200.000000000015
pellet_flows_33_17: 200.000000000015
pellet_flows_35_17: 200.000000000015
pellet_flows_36_15: 78.16324382024101
pellet_flows_36_23: 99.99999999995498
pellet_flows_38_10: 105.54470981746829
pellet_flows_38_17: 94.45529018253171
pellet_flows_39_10: 200.000000000015
pellet_flows_40_23: 200.000000000015
pellet_flows_47_15: 184.3128871764789
pellet_flows_47_48: 6.873832891773901e-10
pellet_flows_49_15: 199.99999999999997
pellet_flows_50_15: 200.000000000015


In [49]:
constraints = model.getConstrs()

In [50]:
import sys
original_stdout = sys.stdout
output_file = "model_display.txt"
with open(output_file, "w") as file:
    sys.stdout = file

    # Display information about the model
    model.display()

# Reset the standard output
sys.stdout = original_stdout
print(f"Model display exported to {output_file}")

Model display exported to model_display.txt
